No knock out - Biomass

In [1]:
#the plan for iCHOv1_K1 at HP condition
#knock out all genes, one at a time:
    #1. maximise biomass, then set biomass to simulated value (check IgG)
    #2. minimise essential amino acids, then set them to simulated value
    #3. minimise glucose and non-essential amino acids, then set them to simulated value

In [2]:
import cobra
from pathlib import Path
#library needed to upload the GeMs
from cobra.io import read_sbml_model
#library needed to simulate gene removal
from cobra.flux_analysis import (single_gene_deletion, single_reaction_deletion)

import pandas as pd
import numpy as np

import seaborn as sns

In [3]:
# declaring the path where the files reside; the same folder as the jupyter notebook
data_dir        = Path(".")
data_dir        = data_dir.resolve()
iCHOv1_K1_path  = data_dir / "iCHOv1_K1_final.xml"
model = read_sbml_model(str(iCHOv1_K1_path.resolve()))
#checking the number of reactions, metabolites and genes
n_genes     = len(model.genes)
print(n_genes)
n_reactions = len(model.reactions)
print(n_reactions)
print(len(model.metabolites))

1298
4723
2773


In [4]:
#creating the results pandas dataframe

#1298 rows x (gene + Biomass_p + 12 essential amino acids + glucose + 5 non-essential amino acids)
#1298 x 20

results = pd.DataFrame(0, index=range(1), 
                       columns=["Biomass_p", 
                                "Arginine", "Cysteine", "Histidine", "Isoleucine", "Leucine", "Lysine", 
                                "Methionine", "Phenylalanine", "Proline", "Threonine", "Tryptophan", "Valine",
                                "Glucose", "Asparagine", "Aspartate", "Glutamine", "Serine", "Tyrosine"] )
                                # "IgG"] )

results

,Biomass_p,Arginine,Cysteine,Histidine,Isoleucine,Leucine,Lysine,Methionine,Phenylalanine,Proline,Threonine,Tryptophan,Valine,Glucose,Asparagine,Aspartate,Glutamine,Serine,Tyrosine
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
#setting up the model with the experimental constraints from HP
#regarding constraints:
#the goal is to have the nutrient consumption to be 
#equal to or smaller than the HP condition
#so setting up the upper bound of consumed nutrients as zero
#and the lower bounds using the measurement

#these chemicals had two bounds set up by default in the model
#for AA and glucose, values were changed to the HP condition for the lower bound
#for the other chemicals, since the model used lower exponential condition value for both bounds
# ==> HP condition was used for both
nutrients_two_b_dict = {"EX_arg_L_e_":(-0.01103321,0), "EX_cys_L_e_":(-0.00521978,0), "EX_his_L_e_":(-0.006457565,0), 
                        "EX_ile_L_e_":(-0.011808118,0), "EX_leu_L_e_":(-0.020184502,0), "EX_lys_L_e_":(-0.012583026,0), 
                        "EX_met_L_e_":(-0.004797048,0), "EX_phe_L_e_":(-0.006457565,0), "EX_pro_L_e_":(-0.011623616,0), 
                        "EX_thr_L_e_":(-0.015313653,0), "EX_trp_L_e_":(-0.003173432,0), "EX_val_L_e_":(-0.015092251,0), 
                        "EX_glc_e_":(-0.271070111,0),
                        "EX_asn_L_e_":(-0.074723247,0), "EX_asp_L_e_":(-0.025166052,0), 
                        "EX_gln_L_e_":(-0.002767528,0), "EX_ser_L_e_":(-0.043062731,0), "EX_tyr_L_e_":(-0.005276753,0),
                        #other nutrients
                        "EX_ala_L_e_":(0.023726937,0.023726937), "EX_glu_L_e_":(-0.012398524,-0.012398524), "EX_gly_e_":(0.00697417,0.00697417), 
                        "EX_lac_L_e_":(0.135608856,0.135608856), "EX_nh4_e_":(0.043763838,0.043763838), 
                        "EX_o2_e_":(-1.127472527,-1.127472527)}
                        # "DM_igg_g_": (2.02e-05,2.02e-05)}

#extracting essential nutrients from the previous dict into a seperate dict
essential_nutrients_dict = dict( list(nutrients_two_b_dict.items()) [0:12])
# print(essential_nutrients_dict)

#extracting non-essential nutrients from the previous dict into a seperate dict
non_essential_nutrients_dict = dict( list(nutrients_two_b_dict.items()) [12:18])
# print(non_essential_nutrients_dict)

#these elements have 0 bound or lower bound set by default
nutrients_lower_b_dict = {"EX_ac_e_":(0.0000738007),"EX_chol_e_":(-0.003099631), "EX_cit_e_": 0.002214022, "EX_for_e_": 0.012583026,
                        "EX_glyc_e_": 0.015129151, "EX_mal_L_e_": 0.001660517, "EX_pyr_e_": -0.010774908,
                        "EX_succ_e_": 0.000405904}

nutrients_lower_b_default_dict = {"EX_ac_e_":0,"EX_chol_e_":0, "EX_cit_e_": 0, "EX_for_e_": 0,
                                "EX_glyc_e_": 0, "EX_mal_L_e_": 0, "EX_pyr_e_": 0,
                                "EX_succ_e_": 0}

#these are the same for HP and lower exponential 
# "EX_h2o_e_": -1000, "EX_hco3_e_": -1000, "EX_pi_e_": -1000, "EX_so4_e_": -1000,
# "SK_Asn_X_Ser_Thr_r_":(-0.1), "SK_Ser_Thr_g_":(-0.1), "SK_Tyr_ggn_c_":(-0.1), 
# "SK_pre_prot_r_":(-0.1)

#setting up the bounds of the exchange reactions
for k,v in nutrients_two_b_dict.items():
    # print(k)
    model.reactions.get_by_id(k).bounds=v
    # print(model.reactions.get_by_id(k).bounds)

for k,v in nutrients_lower_b_dict.items():
    # print(k)
    if model.reactions.get_by_id(k).lower_bound > model.reactions.get_by_id(k).upper_bound:
        model.reactions.get_by_id(k).lower_bound = model.reactions.get_by_id(k).upper_bound
    else:
        model.reactions.get_by_id(k).lower_bound=v
        # print(model.reactions.get_by_id(k).lower_bound)

In [10]:
#1. maximising biomass, then setting the obtained value

# #unconstraining IgG
# model.reactions.get_by_id("DM_igg_g_").bounds=(0,1000)

model.objective       = model.problem.Objective(model.reactions.biomass_cho_producing.flux_expression, direction='max')
solution              = model.optimize()
simulated_biomass     = solution.objective_value
if solution.status    == "optimal":
    print("biomass value is {}".format(simulated_biomass) )
    results.iloc[0,0] = round(simulated_biomass,4)
else:
    results.iloc[0,0] = None 

# #collecting IgG value
# igg_value          = solution.fluxes.DM_igg_g_
# results.iloc[i,20] = igg_value
# print(results.iloc[i,20]) 

#2. minimising essential amino acids, then setting the obtained values 
column             = 1
simulated_ess_nutr = []
for k,v in essential_nutrients_dict.items():
    #setting obtained biomass
    model.reactions.get_by_id("biomass_cho_producing").bounds = (simulated_biomass,simulated_biomass)
    # print(model.reactions.get_by_id("biomass_cho_producing").bounds)

    print(k)
    
    #unconstraining everything
    for k2,v2 in nutrients_two_b_dict.items():
        # print(k2)
        model.reactions.get_by_id(k2).bounds=(-1000,1000)
        # print(model.reactions.get_by_id(k2).bounds)

    for k3,v3 in nutrients_lower_b_default_dict.items():
        # print(k3)
        if v3 > model.reactions.get_by_id(k3).upper_bound:
            model.reactions.get_by_id(k3).bounds = model.reactions.get_by_id(k3).upper_bound
        else:
            model.reactions.get_by_id(k3).lower_bound=v3
            # print(model.reactions.get_by_id(k3).lower_bound)
    
    #optimising
    model.objective            = k
    model.objective.direction  = "max"
    # print(model.objective)
    solution                   = model.optimize()
    simulated_ess_nutr.append(solution.objective_value)
    # print(simulated_ess_nutr)
    if solution.status         == "optimal":
        print("ess. nutrient value is {}".format(simulated_ess_nutr[column-1]) )
        results.iloc[0,column] = round(simulated_ess_nutr[column-1],4)
        column +=1
    else:
        results.iloc[0,column] = None
        column +=1
        
#set new constraints

#3. minimising glucose and non-essential amino acids
#the first step of unconstraining differs between essential and non-essential
#nutrients. For essential, everything is unconstrained beforehand except for 
#growth and productivity.
#for non-essential nutrients on the otherhand, everything is constrained, including
#the growth and essential nutrient values obtained above, and the other non-essential
#nutrient not being minimised at the moment. 

#resetting experimental bounds
#setting up the bounds of the exchange reactions
for k,v in nutrients_two_b_dict.items():
    # print(k)
    model.reactions.get_by_id(k).bounds      = v
    # print(model.reactions.get_by_id(k).bounds)

for k,v in nutrients_lower_b_dict.items():
    print(k)
    print(v)
    print(model.reactions.get_by_id(k).lower_bound)          
    print(model.reactions.get_by_id(k).upper_bound)     
    ku = model.reactions.get_by_id(k).upper_bound
    if v > ku:
        model.reactions.get_by_id(k).bounds = (v,v)
    else:
        model.reactions.get_by_id(k).lower_bound = v

#setting up the simulated essential nutrients
count=0
for ess_nutr in essential_nutrients_dict:
    # print(ess_nutr)
    model.reactions.get_by_id(ess_nutr).bounds = (simulated_ess_nutr[count],simulated_ess_nutr[count])
    # print(model.reactions.get_by_id(ess_nutr).bounds)
    count +=1

#optimising for non-essential nutrients
column=13
simulated_non_ess_nutr = []
for k,v in non_essential_nutrients_dict.items():
    model.objective            = k
    model.objective.direction  = "max"
    print(model.objective)
    solution                   = model.optimize()
    simulated_non_ess_nutr.append(solution.objective_value)
    # print(simulated_non_ess_nutr)
    if solution.status         == "optimal":
        print("non ess. nutrient value is {}".format(simulated_non_ess_nutr[column-13]) )
        results.iloc[0,column] = round(simulated_non_ess_nutr[column-13],4)
        column +=1
    else:
        results.iloc[0,column] = None 
        column +=1

biomass value is 0.023071205141557468
EX_arg_L_e_
ess. nutrient value is -0.007566783134285342
EX_cys_L_e_
ess. nutrient value is -0.0028401982818436933
EX_his_L_e_
ess. nutrient value is -0.003145055473516245
EX_ile_L_e_
ess. nutrient value is -0.006786423637496714
EX_leu_L_e_
ess. nutrient value is -0.013635227481460526
EX_lys_L_e_
ess. nutrient value is -0.01258302599999998
EX_met_L_e_
ess. nutrient value is -0.003235822859438409
EX_phe_L_e_
ess. nutrient value is -0.005967145699783506
EX_pro_L_e_
ess. nutrient value is -0.007727539079983409
EX_thr_L_e_
ess. nutrient value is -0.008881516436714436
EX_trp_L_e_
ess. nutrient value is -0.0010729817210828898
EX_val_L_e_
ess. nutrient value is -0.010947329037137944
EX_ac_e_
7.38007e-05
0
1000.0
EX_chol_e_
-0.003099631
0
1000.0
EX_cit_e_
0.002214022
0
1000.0
EX_for_e_
0.012583026
0
1000.0
EX_glyc_e_
0.015129151
0
1000.0
EX_mal_L_e_
0.001660517
0
1000.0
EX_pyr_e_
-0.010774908
0
1000.0
EX_succ_e_
0.000405904
0
1000.0
Maximize
1.0*EX_glc_e_ 

In [ ]:
model.objective       = model.problem.Objective(model.reactions.biomass_cho_producing.flux_expression, direction='max')
solution              = model.optimize()
simulated_biomass     = solution.objective_value
model.summary()


In [11]:
results

,Biomass_p,Arginine,Cysteine,Histidine,Isoleucine,Leucine,Lysine,Methionine,Phenylalanine,Proline,Threonine,Tryptophan,Valine,Glucose,Asparagine,Aspartate,Glutamine,Serine,Tyrosine
0,0.0231,-0.0076,-0.0028,-0.0031,-0.0068,-0.0136,-0.0126,-0.0032,-0.006,-0.0077,-0.0089,-0.0011,-0.0109,-0.2044,-0.0514,0.0,0,0,-0.0046


In [ ]:
# model.metabolites."igg"
model.metabolites.get_by_id("igg_g")

In [12]:
#Exporting to csv
results.to_csv('no_knockouts_results.csv')

In [ ]:
#extracting unique values
unique_biomass_p        = pd.unique(results.iloc[:,1].round(4))
#output is np array
print(unique_biomass_p)

#sorting
sorted_unique_biomass_p = np.sort(unique_biomass_p)
print(sorted_unique_biomass_p)
length                  = len(unique_biomass_p)
print(length)
print(sorted_unique_biomass_p[length-10:length])

In [ ]:
sns.set_theme()
data = results
data


In [ ]:
sns.relplot(
    data=data,
    x="Gene"  , y="Biomass_p", 
    # col="time",
    # hue="smoker", style="smoker", size="size",
)

In [ ]:
sns.displot(data=data, x="Biomass_p", col="Gene", kde=True)

In [ ]:
sns.histplot(data=data, x="IgG")